In [ ]:
def make_word_list(word_list):    #This function will split the document in words .
    W=[]
    w=''
    for word in word_list: 
       if ((word!=' ')and(word!='.')and(word!=']')and(word!='\n')and(word!='-')and(word!='—')and(word!='?')and(word!='"')and(word!='…')and(word!='/')):
          w=w+word 
       elif ((w!='') ):
          W=W+[w]  
          w=''
    if ((w!='') ):
        W=W+[w]    
    return(W)



def stemming(wor):    #This function will do stemming using nltk library                       
    from nltk.stem import PorterStemmer
    ps = PorterStemmer() 
    return(ps.stem(wor))




def case_folding(wo):     #This function will do case folding .
    return(wo.lower())


def remove_punctuation(w):  #This function will remove punctuations from  words 
    import re
    w=re.sub('(\d)',"",w)
    w=w.replace("—","")
    w=w.replace("_","")
    w=w.replace("?","")
    w=w.replace(".","")
    w=w.replace("`","")
    w=w.replace(",","")
    w=w.replace("[","")
    w=w.replace("]","")
    w=w.replace("â€”","")
    w=w.replace(":","")
    w=w.replace(";","")
    w=w.replace("-","")
    w=w.replace("…","")
    w=w.replace("Â","")
    w=w.replace("/","")
    w=w.replace("â","")
    w=w.replace("'","")
    w=w.replace("–","")
    w=w.replace('"',"")
    w=w.replace("$","")
    w=w.replace("â–","")
    w=w.replace("%","")
    w=w.replace("(","")
    w=w.replace("&","")
    w=w.replace(")","")
    w=w.replace("ã©","")
    w=w.replace("!","")
    return(w)




def pre_processing(w):    #This function will call all preprocessing functions
    w=case_folding(w)
    w=remove_punctuation(w)
    w=stemming(w)
    return(w)



def make_word_list_query(word_list):  #This function will divide the query in words .
    W=[]
    w=''
    for word in word_list: 
       if ((word!=' ')and(word!='.')and(word!=']')and(word!='\n')and(word!='-')and(word!='—')and(word!='?')and(word!='"')and(word!='…')):
          w=w+word 
       elif ((w!='') ):
          W=W+[w]  
          w=''
    if ((w!='') ):
        W=W+[w]
    return(W)




def read_stopwords():  #This function will return list of all stop words .
  stopword=[]
  f=open("Stopword-List.txt","r")
  stopwordlist=f.readlines()
  f.close()
  for l in stopwordlist:
     stopword=stopword+l.split()
  return(stopword)



def get_doc_id(fn):  #This function will take document name and will give ID of that document
    if(fn[8]=='.'):
        return(fn[7])
    else:
        return(fn[7]+fn[8])
    
    

def Make_Indexs():    #This function will read all documents and will make Inverted and Position Index . 
  import os
  directory=os.getcwd() 
  Inverted_Index={}
  Position_Index={}
  for filename in os.listdir(directory):
    if ((filename.endswith(".txt"))and(filename[0]=='s')):
            i=-1
            f = open(filename)
            lines = f.readlines()
            f.close()
            wordlist=make_word_list(lines[1])
            for word in wordlist :
                  i=i+1
                  word=case_folding(word)
                  word=remove_punctuation(word)
                  if(word==''):
                    continue
                  stopword=read_stopwords()  
                  if word not in stopword: 
                     word=stemming(word)   
                     if word not in Inverted_Index:
                        Inverted_Index[word]=[]
                     if get_doc_id(filename) not in Inverted_Index[word]:   
                       Inverted_Index[word]=Inverted_Index[word]+[get_doc_id(filename)]  
                     if word not in Position_Index:
                       Position_Index[word] = {get_doc_id(filename) : [i]}
                     elif get_doc_id(filename) in Position_Index[word]:
                       Position_Index[word][get_doc_id(filename)] += [i]
                     else:
                       Position_Index[word][get_doc_id(filename)] = [i]   
    else:
       continue   
    
    
  return(Inverted_Index,Position_Index)

    
    

def simple_query(w,Inverted_Index,Position_Index): #This function will solve one word phrasal query
    result=[]
    w=pre_processing(w)
    if w in Inverted_Index:
        result=Inverted_Index[w]
    return(result)




def phrasal_query(w,Inverted_Index,Position_Index):  #This function will solve bi word phrasal query
    result=[] 
    for i in range(0,len(w)):
          w[i]=pre_processing(w[i])
    if ( (w[0] in Inverted_Index) and (w[1] in Inverted_Index) ):        
       for i in range (0,56):
          if ( (str(i) in Inverted_Index[w[0]]) and (str(i) in Inverted_Index[w[1]]) ):
              left=0
              right=0
              while((left<(len(Position_Index[w[0]][str(i)]))) and (right<(len(Position_Index[w[1]][str(i)])))):
                  if(int(Position_Index[w[1]][str(i)][right])-int(Position_Index[w[0]][str(i)][left])==1):
                      result.append(str(i))
                      break
                  elif (int(Position_Index[w[1]][str(i)][right])>int(Position_Index[w[0]][str(i)][left])):
                      left=left+1
                  else:
                      right=right+1

    return(result) 






def proximity_query(w,Inverted_Index,Position_Index):  #This function will solve proximity query
    result=[]
    k=w[-1]
    for i in range(0,len(w)):
          w[i]=pre_processing(w[i])
    if ( (w[0] in Inverted_Index) and (w[1] in Inverted_Index) ):        
       for i in range (0,56):
          if ( (str(i) in Inverted_Index[w[0]]) and (str(i) in Inverted_Index[w[1]]) ):
              left=0
              right=0
              while((left<(len(Position_Index[w[0]][str(i)]))) and (right<(len(Position_Index[w[1]][str(i)])))):
                  if(int(Position_Index[w[1]][str(i)][right])-int(Position_Index[w[0]][str(i)][left])==int(k)+1):
                      result.append(str(i))
                      break
                  elif (int(Position_Index[w[1]][str(i)][right])>int(Position_Index[w[0]][str(i)][left])):
                      left=left+1
                  else:
                      right=right+1
            
    return(result)




def Complement(w,Inverted_Index,Position_Index):#This function will return Complement of a list  
    result=[]
    if (type(w) is str):
      if(len(Inverted_Index[w])==0):
         Inverted_Index[w]=[]  
      for i in  range(0,56):
         if str(i) not in Inverted_Index[w]:
            result.append(str(i))
    else:
       for i in  range(0,56):
         if str(i) not in w:
            result.append(str(i)) 
    return(result)




def Intersection(w1,w2,w3,Inverted_Index,Position_Index):# This will function will return Intersection of two or three lists
    l1=l2=l3=result=[]
    if(type(w1) is str):
        if(len(Inverted_Index[w1])==0):
           Inverted_Index[w1]=[]
        l1=Inverted_Index[w1]
    else:
        l1=w1
    if(type(w2) is str):
        if(len(Inverted_Index[w2])==0):
           Inverted_Index[w2]=[]
        l2=Inverted_Index[w2]
    else:
        l2=w2
    if(type(w3) is str):
        if(len(Inverted_Index[w3])==0):
           Inverted_Index[w3]=[]
        l3=Inverted_Index[w3]
    elif (type(w3)is list):
        l3=w3
    if(type(w3) is bool):
        for i in l1:
          if i in l2:
            result.append(i)            
    else:
        for i in l1:
          if ((i in l2) and (i in l3)):
            result.append(i)                       
    return(result)



def Union(w1,w2,w3,Inverted_Index,Position_Index):#This function will return union of two or three  lists
    l1=l2=l3=result=[]
    if(type(w1) is str):
        if(len(Inverted_Index[w1])==0):
          Inverted_Index[w1]=[]
        l1=Inverted_Index[w1]
    else:
        l1=w1
    if(type(w2) is str):
        if(len(Inverted_Index[w2])==0):
          Inverted_Index[w2]=[]
        l2=Inverted_Index[w2]
    else:
        l2=w2
    if(type(w3) is str):
        if(len(Inverted_Index[w3])==0):
           Inverted_Index[w3]=[]
        l3=Inverted_Index[w3]
    elif (type(w3)is list):
        l3=w3
    if(type(w3) is bool):
        for i in l1:
          result.append(i)
        for i in l2:
           if i not in result:
              result.append(i)
    else:
        for i in l1:
          result.append(i)
        for i in l2:
           if i not in result:
              result.append(i)
        for i in l3:
           if i not in result:
              result.append(i)        
    return(result)    





def boolean_query(w,Inverted_Index,Position_Index): #This function will solve simple and complex boolean  query
    result=[]
    while ('(' in w):
        result=[]
        i1=w.index('(')
        i2=w.index(')')
        if(i2-i1==3):
            if(type(w[i1+2] is str)):
               w[i1+2]=pre_processing(w[i1+2])
            result=(Complement(w[i1+2],Inverted_Index,Position_Index))
        elif(i2-i1==4):
            if(type(w[i1+1] is str)):
              w[i1+1]=pre_processing(w[i1+1])
            if(type(w[i1+3] is str)):
              w[i1+3]=pre_processing(w[i1+3])
            if(w[i1+2]=='AND'):
                result=(Intersection(w[i1+1],w[i1+3],True,Inverted_Index,Position_Index))
            else:
                result=(Union(w[i1+1],w[i1+3],True,Inverted_Index,Position_Index))
        elif(i2-i1==6):
            if(type(w[i1+1] is str)):
               w[i1+1]=pre_processing(w[i1+1])
            if(type(w[i1+3] is str)):
               w[i1+3]=pre_processing(w[i1+3])
            if(type(w[i1+5] is str)):
              w[i1+5]=pre_processing(w[i1+5])            
            if(w[i1+2]=='AND'):
                result=(Intersection(w[i1+1],w[i1+3],w[i1+5],Inverted_Index,Position_Index))
            else:
                result=(Union(w[i1+1],w[i1+3],w[i1+5],Inverted_Index,Position_Index))
                
        for i in range(0,i2-i1+1):
            w.pop(i1)
        w.insert(i1,result)
        
    if(len(w)!=1):
        if(len(w)==2):
            if(type(w[1]) is str):
                w[1]=pre_processing(w[1])
            w.pop(0)    
            w.insert(0,Complement(w[0],Inverted_Index,Position_Index))
        elif(len(w)==3):
            if(type(w[0]) is str):
                w[0]=pre_processing(w[0])
            if(type(w[2]) is str):
                w[2]=pre_processing(w[2])
            if(w[1]=='AND'):
                w.pop(1)
                w[0]=Intersection(w[0],w[1],True,Inverted_Index,Position_Index)
                w.pop(1)
            else:
                w.pop(1)
                w[0]=Union(w[0],w[1],True,Inverted_Index,Position_Index)
                w.pop(1)
        else:
            if(type(w[0] is str)):
                w[0]=pre_processing(w[0])
            if(type(w[2] is str)):
                w[2]=pre_processing(w[2])
            if(type(w[4] is str)):
                w[4]=pre_processing(w[4])
            if(w[1]=='AND'):
                w[0]=Intersection(w[0],w[2],w[4],Inverted_Index,Position_Index)
                w.pop(1)
                w.pop(1)
                w.pop(1)
                w.pop(1)
            else:
                w[0]=Union(w[0],w[2],w[4],Inverted_Index,Position_Index)
                w.pop(1)
                w.pop(1)
                w.pop(1)
                w.pop(1)                
    return(w[0])
    
    

    
    
    
def Query(query,Inverted_Index,Position_Index):#This will take query and will call the relevant query function
  query=query+" "
  result=[]
  quer=make_word_list_query(query)
  if len(quer)==1:
    result=simple_query(quer[0],Inverted_Index,Position_Index)
  elif ((len(quer)==2) and ("NOT" not in quer) ):
    result=phrasal_query(quer,Inverted_Index,Position_Index)
  elif ( "/" in quer ):
    result=proximity_query(quer,Inverted_Index,Position_Index)
  else :
    result=boolean_query(quer,Inverted_Index,Position_Index)
  return(result)



#Following code is for GUI and taking user query as input 

import tkinter
from tkinter import ttk

window=tkinter.Tk()
window.title("Boolean Retrieval Model")

labelone=ttk.Label(window,text="Enter Query : ")
labelone.grid(row = 0, column = 0)

labeltwo=ttk.Label(window,text="Result of Query by Boolean Retrieval Model is : ")
labeltwo.grid(row = 2, column = 0)


inp=tkinter.StringVar()


userentry=ttk.Entry(window,width=50,textvariable = inp)
userentry.grid(row = 0 , column = 1)

Inverted_Index,Position_Index=Make_Indexs()



def action():
   
   result=Query(inp.get(),Inverted_Index,Position_Index)
   if(len(result)==0):
       labelthree=ttk.Label(window,text='No Result Found .')
   else:
       labelthree=ttk.Label(window,text=result)
   labelthree.grid(row = 2, column = 1)

   labelthree.after(10000,lambda: labelthree.destroy())


btn = ttk.Button(window,text="Submit",command=action)
btn.grid(row = 0,column = 2)



window.mainloop()




Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Essam\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-4-c1178d7ad373>", line 418, in action
    result=Query(inp.get(),Inverted_Index,Position_Index)
  File "<ipython-input-4-c1178d7ad373>", line 386, in Query
    result=boolean_query(quer,Inverted_Index,Position_Index)
  File "<ipython-input-4-c1178d7ad373>", line 354, in boolean_query
    if(type(w[4] is str)):
IndexError: list index out of range
